In [1]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
from tqdm import tqdm
from pprint import pprint
from tqdm import tqdm
import pickle

In [2]:
def scrapweb(url,driver):
    driver.get(url)
    driver.implicitly_wait(5)
    soup = BeautifulSoup(driver.page_source,'lxml')
    listprop = driver.find_elements_by_class_name('strip_list')
    obj_dict={}
    dfs=pd.DataFrame()
    for obj in listprop:
#         title=obj.find_elements_by_class_name('strip_list wow fadeIn animated')
        reserve_price=obj.find_element_by_class_name('rating').text.replace('Reserve Price: ','')
        reserve_price=reserve_price.replace(',','')
        auction_id =obj.find_element_by_tag_name('small').text.replace('AUCTION ID:','')
        title=obj.find_element_by_tag_name('h5').text
        desc=obj.find_element_by_tag_name('p').text
        date =obj.find_element_by_class_name('btn_listing').text.replace('Auction Date: ','')
        li=obj.find_element_by_tag_name("ul").find_elements_by_tag_name("li")
        li.pop(0)
        for l in li:
            url=l.find_element_by_tag_name("a").get_attribute("href")
            obj_dict[auction_id] = {
                 "Title" : title, 
                 "url":url,
                "Description": desc,
                 "Date":date,
                "Auction_id":auction_id,
                "Reserve_Price":reserve_price 
                }
        df=pd.DataFrame(obj_dict).T
    dfs=dfs.append(df)
#     driver.close()
    return dfs

In [3]:
url ='https://www.eauctionsindia.com/search'
driver = webdriver.Chrome()
driver.get(url)
driver.implicitly_wait(5)
soup = BeautifulSoup(driver.page_source,'lxml')
lis =driver.find_elements_by_class_name('page-link')
t =lis[-1]
t.click()
path=(driver.current_url)
print(path)
dfs= pd.DataFrame()
n =int(path.split('/')[-1])
driver.close()
driver = webdriver.Chrome()
print(n)
df=pd.DataFrame()
dfs=pd.DataFrame()
for i in range(1,n):
    new_url=url+"/" +str(i)
    df=scrapweb(new_url,driver)
    dfs=dfs.append(df)
driver.close()

# new_url='https://www.eauctionsindia.com/search/4'
# scrapweb(new_url,driver)
#     dfs.append(df)
# print(dfs)


https://www.eauctionsindia.com/search


In [4]:
dfs

,Title,url,Description,Date,Auction_id,Reserve_Price
28003,"Flat in Belghoria , Kolkata",https://www.eauctionsindia.com/property-detail...,"Flat No. 4 on 2nd Floor, measuring 1060 Sq.ft,...",27-10-2021 05:00 PM,28003,2456100
28002,"House in Bishnupur , Kolkata",https://www.eauctionsindia.com/property-detail...,Land measuring 9.5 Decimals or 5 Kathas 11 Chi...,27-10-2021 05:00 PM,28002,4855000
28001,"Flat in Howrah , Howrah",https://www.eauctionsindia.com/property-detail...,All that piece and parcel of the property Flat...,27-10-2021 05:00 PM,28001,4454000
28000,"Flat in Howrah , Howrah",https://www.eauctionsindia.com/property-detail...,All that piece and parcel of the property Flat...,27-10-2021 05:00 PM,28000,3854000
27999,"Land in 24 Parganas (North) , Kolkata",https://www.eauctionsindia.com/property-detail...,5 Decimals of Land together one Building Land ...,27-10-2021 05:00 PM,27999,2922000
...,...,...,...,...,...,...
27927,"Land in Belapu , Udupi",https://www.eauctionsindia.com/property-detail...,Land to the extent 5 acres and 85 cents in the...,08-10-2021 03:00 PM,27927,124864000
27926,"Land in Dakshina Kannada , Dakshina Kannada",https://www.eauctionsindia.com/property-detail...,Land to the extent 3 acres and 3.25 cents in t...,08-10-2021 03:00 PM,27926,64727000
27925,"Residential Flat in Kidiyoor , Udupi",https://www.eauctionsindia.com/property-detail...,"Residential Flat bearing No. 005, Panchayath D...",08-10-2021 03:00 PM,27925,3456000
27924,"Commercial Property in Indore , Indore",https://www.eauctionsindia.com/property-detail...,"Unit No LG7, LG-19 & LG-20, timer Ground Floor...",30-10-2021 01:00 PM,27924,3062400


In [6]:
new = dfs['Date'].str.split(" ", n = 1, expand = True)
dfs['Date']=new[0]
dfs['Time']=new[1]
print(dfs)

                                              Title  \
 28003                  Flat in Belghoria , Kolkata   
 28002                 House in Bishnupur , Kolkata   
 28001                      Flat in Howrah , Howrah   
 28000                      Flat in Howrah , Howrah   
 27999        Land in 24 Parganas (North) , Kolkata   
...                                             ...   
 27927                       Land in Belapu , Udupi   
 27926  Land in Dakshina Kannada , Dakshina Kannada   
 27925         Residential Flat in Kidiyoor , Udupi   
 27924       Commercial Property in Indore , Indore   
 27923                        Plot in Mhow , Indore   

                                                      url  \
 28003  https://www.eauctionsindia.com/property-detail...   
 28002  https://www.eauctionsindia.com/property-detail...   
 28001  https://www.eauctionsindia.com/property-detail...   
 28000  https://www.eauctionsindia.com/property-detail...   
 27999  https://www.eauctionsindia

In [7]:
dfs

,Title,url,Description,Date,Auction_id,Reserve_Price,Time
28003,"Flat in Belghoria , Kolkata",https://www.eauctionsindia.com/property-detail...,"Flat No. 4 on 2nd Floor, measuring 1060 Sq.ft,...",27-10-2021,28003,2456100,05:00 PM
28002,"House in Bishnupur , Kolkata",https://www.eauctionsindia.com/property-detail...,Land measuring 9.5 Decimals or 5 Kathas 11 Chi...,27-10-2021,28002,4855000,05:00 PM
28001,"Flat in Howrah , Howrah",https://www.eauctionsindia.com/property-detail...,All that piece and parcel of the property Flat...,27-10-2021,28001,4454000,05:00 PM
28000,"Flat in Howrah , Howrah",https://www.eauctionsindia.com/property-detail...,All that piece and parcel of the property Flat...,27-10-2021,28000,3854000,05:00 PM
27999,"Land in 24 Parganas (North) , Kolkata",https://www.eauctionsindia.com/property-detail...,5 Decimals of Land together one Building Land ...,27-10-2021,27999,2922000,05:00 PM
...,...,...,...,...,...,...,...
27927,"Land in Belapu , Udupi",https://www.eauctionsindia.com/property-detail...,Land to the extent 5 acres and 85 cents in the...,08-10-2021,27927,124864000,03:00 PM
27926,"Land in Dakshina Kannada , Dakshina Kannada",https://www.eauctionsindia.com/property-detail...,Land to the extent 3 acres and 3.25 cents in t...,08-10-2021,27926,64727000,03:00 PM
27925,"Residential Flat in Kidiyoor , Udupi",https://www.eauctionsindia.com/property-detail...,"Residential Flat bearing No. 005, Panchayath D...",08-10-2021,27925,3456000,03:00 PM
27924,"Commercial Property in Indore , Indore",https://www.eauctionsindia.com/property-detail...,"Unit No LG7, LG-19 & LG-20, timer Ground Floor...",30-10-2021,27924,3062400,01:00 PM


In [5]:
dfs.to_pickle("eauction_india_property_search_cleaned.pkl")